In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from operator import methodcaller
import os
from sklearn import linear_model
from recallibrated_data_static.util import *
import math
import pathlib
import itertools
import json

In [2]:


subfolders = [ f.name for f in os.scandir(os.getcwd()) if f.is_dir() ]
recallibrated_dir = 'recallibrated_data_moving'
exclusions = ['data_validate', 'recallibrated_data_moving', 'recallibrated_data_static', 'pilot_testing']
error_df = pd.DataFrame(columns=['Name', 'w1_euc', 'w1_vec', 'w1_index', 'w2_euc', 'w2_vec', 'w2_index', 'w3_euc', 'w3_vec', 'w3_index', 
                                         's4_euc', 's4_vec', 's4_index', 'b5_euc', 'b5_vec', 'b5_index', 
                                        'callibrate_euc', 'callibrate_vec', 'callibrate_index'])
error_df_path = recallibrated_dir + '/' +'error_df.csv'

if os.path.exists(error_df_path):
    error_df = pd.read_csv(error_df_path)

for folder in subfolders:
    desktop = pathlib.Path(folder)
    files = [f.name for f in desktop.iterdir() if f.is_file()]
    data_files = []
    callibration_file = 'not_found'
    
    for f in files:
        if "callibrate" in f:
            callibration_file = folder+"/"+f
        elif "csv" in f:
            data_files.append(folder+"/"+f)

    if callibration_file == 'not_found' or folder in exclusions:
        continue
    
    all_files_callibrated = True
    for f in files: 
        all_files_callibrated = all_files_callibrated and os.path.exists(recallibrated_dir + '/' + f[:-4] + '_recal.csv') 

    if all_files_callibrated:        
        if not folder in error_df['Name'].values:
            errors_in_scene = {}
            for f in files:
                df = pd.read_csv(recallibrated_dir + '/' + f[:-4] + '_recal.csv')
                if 'w1' in f:
                    errors_in_scene['w1_euc'] = spatial_euc_errors(df)
                    errors_in_scene['w1_vec'] = spatial_vec_errors(df)
                    errors_in_scene['w1_index'] = df['correction_index'][0]
                elif 'w2' in f:
                    errors_in_scene['w2_euc'] = spatial_euc_errors(df)
                    errors_in_scene['w2_vec'] = spatial_vec_errors(df)
                    errors_in_scene['w2_index'] = df['correction_index'][0]
                elif 'w3' in f:
                    errors_in_scene['w3_euc'] = spatial_euc_errors(df)
                    errors_in_scene['w3_vec'] = spatial_vec_errors(df)
                    errors_in_scene['w3_index'] = df['correction_index'][0]
                elif 's4' in f:
                    errors_in_scene['s4_euc'] = spatial_euc_errors(df)
                    errors_in_scene['s4_vec'] = spatial_vec_errors(df)
                    errors_in_scene['s4_index'] = df['correction_index'][0]
                elif 'b5' in f:
                    errors_in_scene['b5_euc'] = spatial_euc_errors(df)
                    errors_in_scene['b5_vec'] = spatial_vec_errors(df)
                    errors_in_scene['b5_index'] = df['correction_index'][0]
            
                elif 'callibrate' in f:
                    errors_in_scene['callibrate_euc'] = spatial_euc_errors(df)
                    errors_in_scene['callibrate_vec'] = spatial_vec_errors(df)
                    errors_in_scene['callibrate_index'] = df['correction_index'][0]

                
            error_df.loc[len(error_df.index)] = [folder, errors_in_scene['w1_euc'], errors_in_scene['w1_vec'], errors_in_scene['w1_index'], 
                                                    errors_in_scene['w2_euc'], errors_in_scene['w2_vec'], errors_in_scene['w2_index'], 
                                                    errors_in_scene['w3_euc'], errors_in_scene['w3_vec'], errors_in_scene['w3_index'], 
                                                    errors_in_scene['s4_euc'], errors_in_scene['s4_vec'], errors_in_scene['s4_index'], 
                                                    errors_in_scene['b5_euc'], errors_in_scene['b5_vec'], errors_in_scene['b5_index'], 
                                                    errors_in_scene['callibrate_euc'], errors_in_scene['callibrate_vec'], errors_in_scene['callibrate_index'], 
                                                ]
            error_df.to_csv(error_df_path, index= False)
            continue
    
    data_frame = clean_df(pd.read_csv(callibration_file))
    [shif_df, idx] = shifted_df(data_frame)

    [x_regr, y_regr] = regression_models(shif_df)
    coeffs1 = x_regr.coef_
    coeffs2 = y_regr.coef_

    if not os.path.exists(recallibrated_dir):
        os.makedirs(recallibrated_dir)

    errors_in_scene = {}
    all_files = itertools.chain(data_files, [callibration_file])

    for f in all_files:
        if os.path.exists(recallibrated_dir + '/' + f[:-4] + '_recal.csv'):
            continue
        
        df = pd.read_csv(f)
        df = clean_df(df)

        [df, idx] = shifted_df(df)

        
        df['gaze_x_recal'] = df.apply(lambda row: x_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)
        df['gaze_y_recal'] = df.apply(lambda row: y_regr.predict([[row['gaze_vis_x'],row['gaze_vis_y']]])[0], axis = 1)

        gaze_vec = df['gaze_vector'].str.replace('_',',').str.replace('(','[').str.replace(')',']').apply(lambda p: json.loads(p))
        target_vec = df['target_vector'].str.replace('_',',').str.replace('(','[').str.replace(')',']').apply(lambda p: json.loads(p))

        df = df[['frame','path','seconds','gaze_vis_x','gaze_vis_y','gaze_x_recal','gaze_y_recal', 'target_vis_x', 'target_vis_y', 'blinks']]
        df['correction_index'] = [idx for i in np.arange(0, len(df))]
        df['gaze_vector'] = gaze_vec
        df['target_vector'] = target_vec

        f = f.replace(folder, '')
        df.to_csv(recallibrated_dir+f[:-4]+'_recal.csv')

        if 'w1' in f:
            errors_in_scene['w1_euc'] = spatial_euc_errors(df)
            errors_in_scene['w1_vec'] = spatial_vec_errors(df)
            errors_in_scene['w1_index'] = idx
        elif 'w2' in f:
            errors_in_scene['w2_euc'] = spatial_euc_errors(df)
            errors_in_scene['w2_vec'] = spatial_vec_errors(df)
            errors_in_scene['w2_index'] = idx
        elif 'w3' in f:
            errors_in_scene['w3_euc'] = spatial_euc_errors(df)
            errors_in_scene['w3_vec'] = spatial_vec_errors(df)
            errors_in_scene['w3_index'] = idx
        elif 's4' in f:
            errors_in_scene['s4_euc'] = spatial_euc_errors(df)
            errors_in_scene['s4_vec'] = spatial_vec_errors(df)
            errors_in_scene['s4_index'] = idx
        elif 'b5' in f:
            errors_in_scene['b5_euc'] = spatial_euc_errors(df)
            errors_in_scene['b5_vec'] = spatial_vec_errors(df)
            errors_in_scene['b5_index'] = idx
       
        elif 'callibrate' in f:
            errors_in_scene['callibrate_euc'] = spatial_euc_errors(df)
            errors_in_scene['callibrate_vec'] = spatial_vec_errors(df)
            errors_in_scene['callibrate_index'] = idx

        if len(errors_in_scene) == 18:
            error_df.loc[len(error_df.index)] = [folder, errors_in_scene['w1_euc'], errors_in_scene['w1_vec'], errors_in_scene['w1_index'], 
                                                    errors_in_scene['w2_euc'], errors_in_scene['w2_vec'], errors_in_scene['w2_index'], 
                                                    errors_in_scene['w3_euc'], errors_in_scene['w3_vec'], errors_in_scene['w3_index'], 
                                                    errors_in_scene['s4_euc'], errors_in_scene['s4_vec'], errors_in_scene['s4_index'], 
                                                    errors_in_scene['b5_euc'], errors_in_scene['b5_vec'], errors_in_scene['b5_index'], 
                                                    errors_in_scene['callibrate_euc'], errors_in_scene['callibrate_vec'], errors_in_scene['callibrate_index'], 
                                                 ]

    error_df.to_csv(error_df_path, index= False)


C:\Users\satya\AppData\Local\Temp\ipykernel_13728\990374293.py:99: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  gaze_vec = df['gaze_vector'].str.replace('_',',').str.replace('(','[').str.replace(')',']').apply(lambda p: json.loads(p))
C:\Users\satya\AppData\Local\Temp\ipykernel_13728\990374293.py:100: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  target_vec = df['target_vector'].str.replace('_',',').str.replace('(','[').str.replace(')',']').apply(lambda p: json.loads(p))
C:\Users\satya\AppData\Local\Temp\ipykernel_13728\990374293.py:99: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressi

KeyError: 'PathIDX'